<a href="https://colab.research.google.com/github/preetamjumech/pytorch/blob/main/RNN_QA_System_11_02_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
# vocab
vocab = {'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

In [6]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
len(vocab)

324

In [8]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [9]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [12]:
dataset = QADataset(df, vocab)

In [13]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [14]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 10,  75, 208]]) tensor([209])
tensor([[10, 29,  3, 30, 31]]) tensor([32])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([207])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]]) tensor([23])
tensor([[ 42, 137,   2, 138,  39, 139]]) tensor([53])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([268])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([149])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([249])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([91])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([134])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([74])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([307])
tensor([[ 10, 308,   3, 309, 310]]) tensor([311])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) 

In [15]:
import torch.nn as nn

In [16]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [17]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [18]:
learning_rate = 0.001
epochs = 50

In [19]:
model = SimpleRNN(len(vocab))

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 523.888456
Epoch: 2, Loss: 455.839893
Epoch: 3, Loss: 375.339023
Epoch: 4, Loss: 316.031277
Epoch: 5, Loss: 265.686924
Epoch: 6, Loss: 218.942786
Epoch: 7, Loss: 175.841063
Epoch: 8, Loss: 138.063000
Epoch: 9, Loss: 106.828244
Epoch: 10, Loss: 82.194834
Epoch: 11, Loss: 63.892834
Epoch: 12, Loss: 50.042314
Epoch: 13, Loss: 39.486208
Epoch: 14, Loss: 31.989318
Epoch: 15, Loss: 26.331266
Epoch: 16, Loss: 21.734881
Epoch: 17, Loss: 18.183591
Epoch: 18, Loss: 15.397398
Epoch: 19, Loss: 13.240035
Epoch: 20, Loss: 11.448093
Epoch: 21, Loss: 9.981798
Epoch: 22, Loss: 8.684029
Epoch: 23, Loss: 7.741779
Epoch: 24, Loss: 6.867689
Epoch: 25, Loss: 6.187759
Epoch: 26, Loss: 5.562283
Epoch: 27, Loss: 5.020531
Epoch: 28, Loss: 4.566866
Epoch: 29, Loss: 4.159549
Epoch: 30, Loss: 3.803943
Epoch: 31, Loss: 3.490656
Epoch: 32, Loss: 3.207351
Epoch: 33, Loss: 2.966360
Epoch: 34, Loss: 2.733780
Epoch: 35, Loss: 2.540355
Epoch: 36, Loss: 2.356320
Epoch: 37, Loss: 2.184988
Epoch: 38, Loss: 2

In [22]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [23]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [24]:
list(vocab.keys())[7]

'paris'

In [25]:
predict(model, "What is the capital of india?")

delhi


In [26]:
predict(model, "What is 2 + 2?")

I don't know
alexander-graham-bell
